<a href="https://www.kaggle.com/code/faisalafif/data-wrangling-sql-course-final-project?scriptVersionId=142868119" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Import necessary libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
import plotly.express as px # data visualization
from plotly.subplots import make_subplots # data visualization

import warnings 
warnings.filterwarnings('ignore')

## Accessing Data

In [ ]:
customers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
orders = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
order_reviews = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
payments = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv')
order_items = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
products = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
sellers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
geolocation = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
products_eng = pd.read_csv('/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv')

## Data Wrangling : Discovery, Transforming, Cleaning, Enriching, Verifying, and Publishing

https://online.hbs.edu/blog/post/data-wrangling

### Create 1st table and 2nd table by combining orders, customers and payments tables

In [ ]:
orders.info()

In [ ]:
orders.order_status.value_counts()

In [ ]:
# Only orders that have been delivered will be used in the following analysis.

orders_1 = orders[orders['order_status'] == 'delivered'] # Select orders with delivered status
orders_1 = orders_1.reset_index(drop = True) # Reset the column index

# In the following analysis there are 4 columns that will be used from orders table : order_id, customer_id, order_status, order_purchase_timestamp
column_to_be_dropped = ['order_approved_at','order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
orders_1 = orders_1.drop(column_to_be_dropped, axis=1)

# Convert order_purchase_timestamp into date and time data
orders_1['order_purchase_timestamp'] = pd.to_datetime(orders_1['order_purchase_timestamp'])
orders_1.info()

In [ ]:
# Identifying missing values
orders_1.isna().sum().to_frame().reset_index().rename(columns={'index':'column_name', 0:'value'})

In [ ]:
# Identifying duplicates values
orders_1[orders_1.duplicated(keep=False)]

In [ ]:
customers.info()

In [ ]:
# Identifying missing values
customers.isna().sum().to_frame().reset_index().rename(columns={'index':'column_name', 0:'value'})

In [ ]:
# Identifying duplicates values
customers[customers.duplicated(keep=False)]

In [ ]:
# Identifying inconsistencies in the customer_state column
customers.customer_state.value_counts()

In [ ]:
payments.info()

In [ ]:
# Identifying missing values
payments.isna().sum().to_frame().reset_index().rename(columns={'index':'column_name', 0:'value'})

In [ ]:
# Identifying duplicates values
payments[payments.duplicated(keep=False)]

In [ ]:
# Identifying inconsistencies in the payment_type column
payments.payment_type.value_counts()

In [ ]:
payments[payments['payment_type']=='not_defined']

In [ ]:
# Merge orders table and customers table, then stored in revenue_states
revenue_states = pd.merge(orders_1, customers, how='left', on='customer_id')

# Merge revenue_states table and payments table, then stored in revenue_states
revenue_states = pd.merge(revenue_states, payments, how='left', on='order_id')
revenue_states


In [ ]:
revenue_states.info()

In [ ]:
revenue_states['payment_value'].describe(include='all')

In [ ]:
# Identifying missing values
revenue_states.isna().sum().to_frame().reset_index().rename(columns={'index':'column_name', 0:'value'})

In [ ]:
revenue_states[revenue_states.isna().any(axis=1)]

In [ ]:
# Since there is only 4 missing values in 1 row from 100757 rows, removing one row will not significantly affect the analysis.
revenue_states = revenue_states.dropna()

In [ ]:
# Identifying duplicates values
revenue_states[revenue_states.duplicated(keep=False)]

### **1st table : revenue_states table -> Question 1 & 2**

In [ ]:
revenue_states

### Create 2nd table by selecting order_id, order_status, order_purchase_timestamp and customer_state columns from revenue_states

In [ ]:
order_states = revenue_states[['order_id', 'order_status', 'order_purchase_timestamp', 'customer_state']]
order_states

In [ ]:
# Identifying duplicates values
order_states[order_states.duplicated(keep=False)]

In [ ]:
# Removing duplicates values
order_states = order_states.drop_duplicates(keep='first') # Remove duplicate data and keep the first data that appears
order_states = order_states.reset_index(drop = True) # Reset the column index
order_states.info()

### **2nd table : order_states table -> Question 3**

In [ ]:
order_states

### Create 3rd table by combining products, products_eng, revenue_states and order_items tables

In [ ]:
products.info()

In [ ]:
products_eng.info()

In [ ]:
# Merge products table and products_eng table, stored in products_merged
products_merged = pd.merge(products, products_eng, how='left', on='product_category_name')

# Merge revenue_states table and order_items table, stored in revenue_order_items
revenue_order_items = pd.merge(revenue_states, order_items, how='left', on='order_id')

# Merge revenue_order_items table and products_merged table, stored in revenue_order_items
revenue_products = pd.merge(revenue_order_items, products_merged, how='left', on='product_id')

revenue_products.info()

In [ ]:
# In the following analysis there are 11 columns that will be used from previous table : 
# ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'customer_state', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value', 'product_id', 'product_category']

revenue_products = revenue_products[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'customer_state', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value', 'product_id', 'product_category_name_english']]
revenue_products.columns = ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'customer_state', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value', 'product_id', 'product_category']
revenue_products.info()

In [ ]:
# Identifying missing values
revenue_products.isna().sum().to_frame().reset_index().rename(columns={'index':'column_name', 0:'value'})

In [ ]:
a = revenue_products[revenue_products.isna().any(axis=1)]
a

In [ ]:
# There are so many missing values in product category column
# Before removing all of these missing values, consider calculating the proportion of missing value transactions to total transactions.
missing_val_transaction = a.payment_value.sum()
total_transaction = revenue_products.payment_value.sum()
prop = missing_val_transaction / total_transaction * 100
print(f'Proportion of missing value transactions to total transactions = {prop:.2f}%')

In [ ]:
# In order to get clear understanding:
#    (Option 1) all product_category shall be filled or 
#    (Option 2) missing values in product_category shall be removed

# Since the proportion of missing value transactions to total transactions = 1.25%, removing these values still not significantly affect the analysis.
revenue_products = revenue_products.dropna()

In [ ]:
# Identifying duplicates values
revenue_products[revenue_products.duplicated(keep=False)]

In [ ]:
# Removing duplicates values
revenue_products = revenue_products.drop_duplicates(keep='first') # Remove duplicate data and keep the first data that appears
revenue_products = revenue_products.reset_index(drop = True) # Reset the column index
revenue_products.info()

In [ ]:
# Identifying inconsistencies in the product_category column
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
revenue_products.product_category.value_counts()

### **3rd table : revenue_products table -> Question 4 & 5**

In [ ]:
revenue_products

### Create 4th table by selecting order_id, customer_id, customer_state and product_category columns from revenue_products


In [ ]:
order_category = revenue_products[['order_id', 'customer_id', 'customer_state', 'product_category']]
order_category.info()

In [ ]:
# Identifying missing values
order_category.isna().sum().to_frame().reset_index().rename(columns={'index':'column_name', 0:'value'})

In [ ]:
# Identifying duplicates values
order_category[order_category.duplicated(keep=False)]

In [ ]:
# Removing duplicates values
order_category = order_category.drop_duplicates(keep='first') # Remove duplicate data and keep the first data that appears
order_category = order_category.reset_index(drop = True) # Reset the column index
order_category.info()

### **4th table : order_category table -> Question 6**

In [ ]:
order_category

## Data Analysis and Data Visualization

### 1. What are the top 5 revenue-generating states?

In [ ]:
# From table 1, group by 'customer_state' and sum the 'payment_value' for each state
revenue_by_state = revenue_states.groupby('customer_state')['payment_value'].sum().reset_index()

# Sort the summary by highest revenue in descending order
revenue_by_state = revenue_by_state.sort_values(by='payment_value', ascending=False)
revenue_by_state.columns = ['State', 'Revenue']
revenue_by_state

In [ ]:
def calculate_top_5_revenue_percentage(revenue_summary):
    # Sort the revenue summary by highest revenue in descending order
    revenue_summary = revenue_summary.sort_values(by='Revenue', ascending=False)
    
    # Select the top 5 states
    top_5_states = revenue_summary.head(5)
    
    # Calculate the total revenue contributed by the top 5 states
    top_5_revenue = top_5_states['Revenue'].sum()
    
    # Calculate the total revenue
    total_revenue = revenue_summary['Revenue'].sum()
    
    # Calculate the percentage of total revenue contributed by the top 5 states
    percentage = (top_5_revenue / total_revenue) * 100

    return percentage

# Calculate the percentage of top 5 states revenue
top_5_percentage = calculate_top_5_revenue_percentage(revenue_by_state)
print(f'Percentage of total revenue contributed by the top 5 states: {top_5_percentage:.2f}%')

In [ ]:
calculate_top_5_revenue_percentage(revenue_by_state)

In [ ]:
# Create viz1 dataframe for visualization of revenue by state 
viz1 = revenue_by_state

# Create the visualization of revenue by state
fig = px.bar(data_frame=viz1, x='State', y='Revenue', title='What are the top 5 revenue-generating states?')
fig.show()

### 2. How are the revenue trends in those 5 states?

In [ ]:
revenue_states.info()

In [ ]:
# Create function to calculate monthly revenue for specific state per year-month
# Use table 1 (revenue_states), state column and specific states as an input

def calculate_monthly_revenue(revenue_states, state_column, states=None):
    # Filter the DataFrame by state if states are provided
    if states:
        revenue_states = revenue_states[revenue_states[state_column].isin(states)]

    # Extract the year and month from the 'order_purchase_timestamp' column
    revenue_states['year'] = revenue_states['order_purchase_timestamp'].dt.year
    revenue_states['month'] = revenue_states['order_purchase_timestamp'].dt.month

    # Group by year, month, and state (if filtered by state) and sum the revenue
    group_columns = ['year', 'month']
    if state_column in revenue_states.columns:
        group_columns.append(state_column)
    monthly_revenue = revenue_states.groupby(group_columns)['payment_value'].sum().reset_index()
    
    # Sort the result by year and month
    monthly_revenue = monthly_revenue.sort_values(by=['customer_state', 'year', 'month']).reset_index(drop=True)

    # Create a 'year_month' column as a string for the period and return the result
    monthly_revenue['year_month'] = monthly_revenue['year'].astype(str) + '-' + monthly_revenue['month'].astype(str)   
    monthly_revenue.columns = ['year', 'month', 'state', 'revenue', 'year_month']
    return monthly_revenue

# Calculate monthly revenue for specific states
monthly_revenue_SP = calculate_monthly_revenue(revenue_states, 'customer_state', states=['SP'])
monthly_revenue_RJ = calculate_monthly_revenue(revenue_states, 'customer_state', states=['RJ'])
monthly_revenue_MG = calculate_monthly_revenue(revenue_states, 'customer_state', states=['MG'])
monthly_revenue_RS = calculate_monthly_revenue(revenue_states, 'customer_state', states=['RS'])
monthly_revenue_PR = calculate_monthly_revenue(revenue_states, 'customer_state', states=['PR'])

In [ ]:
monthly_revenue_SP

In [ ]:
# Create a subplot with 2 rows and 2 columns
fig = make_subplots(rows=3, cols=2, subplot_titles=('Revenue Trend SP', 'Revenue Trend RJ', 'Revenue Trend MG', 'Revenue Trend RS', 'Revenue Trend PR'))

# Add Plotly Express plots to each subplot
fig.add_trace(px.line(monthly_revenue_SP, x='year_month', y='revenue').data[0], row=1, col=1)
fig.add_trace(px.line(monthly_revenue_RJ, x='year_month', y='revenue').data[0], row=1, col=2)
fig.add_trace(px.line(monthly_revenue_MG, x='year_month', y='revenue').data[0], row=2, col=1)
fig.add_trace(px.line(monthly_revenue_RS, x='year_month', y='revenue').data[0], row=2, col=2)
fig.add_trace(px.line(monthly_revenue_PR, x='year_month', y='revenue').data[0], row=3, col=1)


# Update subplot titles (optional)
fig.update_layout(title_text='How are the revenue trends in those 5 states?',
                  autosize=False,
                  width=1000,
                  height=1200,)

# Show the plot
fig.show()

### 3. How are the sales trends (number of orders) in those 5 states?

In [ ]:
order_states.info()

In [ ]:
# Create function to calculate monthly orders for specific state per year-month
# Use table 2 (order_states), state column and specific states as an input

def calculate_monthly_orders(order_states, state_column, states=None):
    # Filter the DataFrame by state if states are provided
    if states:
        order_states = order_states[order_states[state_column].isin(states)]

    # Extract the year and month from the 'order_purchase_timestamp' column
    order_states['year'] = order_states['order_purchase_timestamp'].dt.year
    order_states['month'] = order_states['order_purchase_timestamp'].dt.month

    # Group by year, month, and state (if filtered by state) and count the order_id
    group_columns = ['year', 'month']
    if state_column in order_states.columns:
        group_columns.append(state_column)
    monthly_order = order_states.groupby(group_columns)['order_id'].count().reset_index()
    
    # Sort the result by year and month
    monthly_order = monthly_order.sort_values(by=['customer_state', 'year', 'month']).reset_index(drop=True)

    # Create a 'year_month' column as a string for the period and return the result
    monthly_order['year_month'] = monthly_order['year'].astype(str) + '-' + monthly_order['month'].astype(str)
    monthly_order.columns = ['year', 'month', 'state', 'number of orders', 'year_month']
    return monthly_order

# Calculate monthly revenue for specific states
monthly_order_SP = calculate_monthly_orders(order_states, 'customer_state', states=['SP'])
monthly_order_RJ = calculate_monthly_orders(order_states, 'customer_state', states=['RJ'])
monthly_order_MG = calculate_monthly_orders(order_states, 'customer_state', states=['MG'])
monthly_order_RS = calculate_monthly_orders(order_states, 'customer_state', states=['RS'])
monthly_order_PR = calculate_monthly_orders(order_states, 'customer_state', states=['PR'])

In [ ]:
monthly_order_SP

In [ ]:
# Create a subplot with 2 rows and 2 columns
fig = make_subplots(rows=3, cols=2, subplot_titles=('Orders Trend SP', 'Orders Trend RJ', 'Orders Trend MG', 'Orders Trend RS', 'Orders Trend PR'))

# Add Plotly Express plots to each subplot
fig.add_trace(px.line(monthly_order_SP, x='year_month', y='number of orders').data[0], row=1, col=1)
fig.add_trace(px.line(monthly_order_RJ, x='year_month', y='number of orders').data[0], row=1, col=2)
fig.add_trace(px.line(monthly_order_MG, x='year_month', y='number of orders').data[0], row=2, col=1)
fig.add_trace(px.line(monthly_order_RS, x='year_month', y='number of orders').data[0], row=2, col=2)
fig.add_trace(px.line(monthly_order_PR, x='year_month', y='number of orders').data[0], row=3, col=1)


# Update subplot titles (optional)
fig.update_layout(title_text='How are the orders trends (growth of orders) in those 5 states?',
                  autosize=False,
                  width=1000,
                  height=1200)

# Show the plot
fig.show()

In [ ]:
# Group data and count orders and products per state
grouped_data = revenue_products.groupby('customer_state').agg(
    num_orders=pd.NamedAgg(column='order_id', aggfunc='nunique'),
    num_products=pd.NamedAgg(column='product_id', aggfunc='nunique')
).reset_index()

# Filtering using DataFrame.loc[]
top_5_states = ['SP','RJ','MG','RS','PR']
group_of_top_5_states = grouped_data.loc[grouped_data['customer_state'].isin(top_5_states)].reset_index(drop=True)

# Calculate the correlation coefficient
num_orders = group_of_top_5_states.loc[:,'num_orders']
num_products = group_of_top_5_states.loc[:,'num_products']

correlation_matrix = np.corrcoef(num_products, num_orders)
correlation_top_5_states = correlation_matrix[0, 1]

print(group_of_top_5_states)
print("Correlation:", correlation_top_5_states)

In [ ]:
# Create scatter plot for each state with x-axis as 'num_products' and y-axis as 'num_orders'
fig = px.scatter(group_of_top_5_states, x='num_products', y='num_orders', color='customer_state',
                 title='Correlation of Number of Products Sold and Number of Orders')

# Customize the plot (optional)
fig.update_traces(marker=dict(size=12))  # Adjust marker size
fig.update_layout(legend_title_text='State',
                  width=600,
                  height=600)  # Set legend title, width and height

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



# Create a scatter plot using Seaborn
plt.figure(figsize=(8, 6))
sns.scatterplot(data=group_of_top_5_states, x='num_products', y='num_orders', hue='customer_state')

# Add the correlation line using Seaborn's regplot
sns.regplot(data=group_of_top_5_states, x='num_products', y='num_orders', scatter=False, color='green', line_kws={"linewidth": 2})

# Customize the plot (optional)
plt.title('Correlation of Number of Products Sold and Number of Orders for Top 5 States')
plt.xlabel('Number of Products')
plt.ylabel('Number of Orders')
plt.legend(title='State')

# Add the correlation coefficient as text to the plot
plt.text(14000, 39000, f'Correlation: {correlation_top_5_states:.4f}', fontsize=8, color='green')

# Show the plot
plt.show()


In [ ]:
# Calculate correlation coefficient for all states between num_orders and num_products

num_orders_all = grouped_data.loc[:,'num_orders']
num_products_all = grouped_data.loc[:,'num_products']

correlation_matrix = np.corrcoef(num_products_all, num_orders_all)
correlation_all = correlation_matrix[0, 1]

print(grouped_data)
print("Correlation:", correlation_all)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



# Create a scatter plot using Seaborn
plt.figure(figsize=(8, 6))
sns.scatterplot(data=grouped_data, x='num_products', y='num_orders')

# Add the correlation line using Seaborn's regplot
sns.regplot(data=grouped_data, x='num_products', y='num_orders', scatter=False, color='green', line_kws={"linewidth": 2})

# Customize the plot (optional)
plt.title('Correlation of Number of Products Sold and Number of Orders for All States')
plt.xlabel('Number of Products')
plt.ylabel('Number of Orders')

# Add the correlation coefficient as text to the plot
plt.text(12500, 38000, f'Correlation: {correlation_all:.4f}', fontsize=8, color='green')

# Show the plot
plt.show()


### 4. What are the top 10 revenue-generating product categories in each of those 5 states?

In [ ]:
revenue_products.info()

In [ ]:
# Create function to calculate revenue per product category for specific state 
# Use table 3 (revenue_products) and state list as an input
def calculate_revenue_per_product_cat(revenue_data, state_list):
    # Create an empty dictionary to store summaries for each state
    revenue_by_state = {}
    
    # Iterate through each state in the state_list
    for state in state_list:
        # Filter data for the current state
        state_data = revenue_data[revenue_data['customer_state'] == state]
        
        # Group by product category and sum payment values
        revenue_summary = state_data.groupby('product_category')['payment_value'].sum().reset_index()
        
        # Store the summary in the dictionary
        revenue_by_state[state] = revenue_summary
        
    return revenue_by_state

# Accessing revenue summary and calculate revenue proportion for a specific state
def calculate_revenue_proportion(revenue_by_state_dict, state):
    # Access the revenue summary for the specified state
    state_summary = revenue_by_state_dict.get(state)

    if state_summary is not None:
        # Calculate the revenue proportion
        state_summary['revenue_proportion'] = (state_summary['payment_value'] / state_summary['payment_value'].sum()) * 100
    
    state_summary.columns = ['product_category', 'revenue', 'revenue_proportion']
    return state_summary

# Create function to sort revenue per product category by the highest revenue
def sort_by_highest_revenue(revenue_summary):
    # Sort the summary by highest revenue in descending order
    sorted_summary = revenue_summary.sort_values(by='revenue', ascending=False)
    sorted_summary = sorted_summary.head(10).reset_index(drop=True)
    return sorted_summary

# Create function to sort revenue per product category by the lowest revenue
def sort_by_lowest_revenue(revenue_summary):
    # Sort the summary by lowest revenue in ascending order
    sorted_summary = revenue_summary.sort_values(by='revenue', ascending=True)
    sorted_summary = sorted_summary.head(10).reset_index(drop=True)
    return sorted_summary

# Top 5 states with highest revenue
states = ['SP', 'RJ', 'MG', 'RS', 'PR']  

# Calculate revenue per product category for top 5 states
revenue_by_state_dict = calculate_revenue_per_product_cat(revenue_products, states)

In [ ]:
sp_summary = calculate_revenue_proportion(revenue_by_state_dict, 'SP')
highest_sp_summary = sort_by_highest_revenue(sp_summary)
highest_sp_summary

In [ ]:
rj_summary = calculate_revenue_proportion(revenue_by_state_dict, 'RJ')
highest_rj_summary = sort_by_highest_revenue(rj_summary)
highest_rj_summary

In [ ]:
mg_summary = calculate_revenue_proportion(revenue_by_state_dict, 'MG')
highest_mg_summary = sort_by_highest_revenue(mg_summary)
highest_mg_summary

In [ ]:
rs_summary = calculate_revenue_proportion(revenue_by_state_dict, 'RS')
highest_rs_summary = sort_by_highest_revenue(rs_summary)
highest_rs_summary

In [ ]:
pr_summary = calculate_revenue_proportion(revenue_by_state_dict, 'PR')
highest_pr_summary = sort_by_highest_revenue(pr_summary)
highest_pr_summary

In [ ]:
# Create a subplot with 2 rows and 2 columns
fig = make_subplots(rows=3, cols=2, subplot_titles=('Top 10 revenue-generating product SP', 'Top 10 revenue-generating product RJ', 'Top 10 revenue-generating product MG', 'Top 10 revenue-generating product RS', 'Top 10 revenue-generating product PR'))

# Add Plotly Express plots to each subplot
fig.add_trace(px.bar(highest_sp_summary, x='product_category', y='revenue').data[0], row=1, col=1)
fig.add_trace(px.bar(highest_rj_summary, x='product_category', y='revenue').data[0], row=1, col=2)
fig.add_trace(px.bar(highest_mg_summary, x='product_category', y='revenue').data[0], row=2, col=1)
fig.add_trace(px.bar(highest_rs_summary, x='product_category', y='revenue').data[0], row=2, col=2)
fig.add_trace(px.bar(highest_pr_summary, x='product_category', y='revenue').data[0], row=3, col=1)


# Update subplot titles (optional)
fig.update_layout(title_text='What are the top 10 revenue-generating product categories in each of those 5 states?',
                  autosize=False,
                  width=1000,
                  height=1200)

# Show the plot
fig.show()

### 5. What are the 10 product categories with the lowest revenue in each of those 5 states?

In [ ]:
lowest_sp_summary = sort_by_lowest_revenue(sp_summary)
lowest_sp_summary

In [ ]:
lowest_rj_summary = sort_by_lowest_revenue(rj_summary)
lowest_rj_summary

In [ ]:
lowest_mg_summary = sort_by_lowest_revenue(mg_summary)
lowest_mg_summary

In [ ]:
lowest_rs_summary = sort_by_lowest_revenue(rs_summary)
lowest_rs_summary

In [ ]:
lowest_pr_summary = sort_by_lowest_revenue(pr_summary)
lowest_rs_summary

In [ ]:
# Create a subplot with 2 rows and 2 columns
fig = make_subplots(rows=3, cols=2, subplot_titles=('10 product categories with the lowest revenue SP', '10 product categories with the lowest revenue RJ', '10 product categories with the lowest revenue MG', '10 product categories with the lowest revenue RS', '10 product categories with the lowest revenue PR'))

# Add Plotly Express plots to each subplot
fig.add_trace(px.bar(lowest_sp_summary, x='product_category', y='revenue').data[0], row=1, col=1)
fig.add_trace(px.bar(lowest_rj_summary, x='product_category', y='revenue').data[0], row=1, col=2)
fig.add_trace(px.bar(lowest_mg_summary, x='product_category', y='revenue').data[0], row=2, col=1)
fig.add_trace(px.bar(lowest_rs_summary, x='product_category', y='revenue').data[0], row=2, col=2)
fig.add_trace(px.bar(lowest_pr_summary, x='product_category', y='revenue').data[0], row=3, col=1)


# Update subplot titles (optional)
fig.update_layout(title_text='What are the 10 product categories with the lowest revenue in each of those 5 states?',
                  autosize=False,
                  width=1000,
                  height=1200)

# Show the plot
fig.show()

### 6. What are the top 10 most-ordered product categories in those five states?

In [ ]:
order_category.info()

In [ ]:
# Create pivot table for summarizing number of orders per category (row) for each state (column)
orders_pivot = pd.crosstab(order_category['product_category'], order_category['customer_state'])

In [ ]:
# Create function to calculate orders per product category for specific state 
# Use orders_pivot table and state list as an input

def select_top_10_categories_for_states(pivot_table, states):
    # Create an empty dataframe to store the top 10 product categories for each state
    top_10_categories_by_state = pd.DataFrame(columns=['State', 'Product_Category', 'Total_Orders']) 
    
    # Iterate through each state in the specified list
    for state in states:
        # Sort the products for the current state in descending order and select the top 10
        top_10_for_state = pivot_table[state].nlargest(10)
        
        # Iterate through the top 10 product categories for the current state
        for category, orders in top_10_for_state.items():
            top_10_categories_by_state = pd.concat([top_10_categories_by_state, pd.DataFrame({
                'State': [state],
                'Product_Category': [category],
                'Total_Orders': [orders]
            })], ignore_index=True)
    
    return top_10_categories_by_state


In [ ]:
SP_cat_orders = select_top_10_categories_for_states(orders_pivot, ['SP'])
SP_cat_orders

In [ ]:
RJ_cat_orders = select_top_10_categories_for_states(orders_pivot, ['RJ'])
RJ_cat_orders

In [ ]:
MG_cat_orders = select_top_10_categories_for_states(orders_pivot, ['MG'])
MG_cat_orders

In [ ]:
RS_cat_orders = select_top_10_categories_for_states(orders_pivot, ['RS'])
RS_cat_orders

In [ ]:
PR_cat_orders = select_top_10_categories_for_states(orders_pivot, ['PR'])
PR_cat_orders

In [ ]:
# Create a subplot with 2 rows and 2 columns 
fig = make_subplots(rows=3, cols=2, subplot_titles=('Top 10 most-ordered product categories in SP', 'Top 10 most-ordered product categories in RJ', 'Top 10 most-ordered product categories in MG', 'Top 10 most-ordered product categories in RS', 'Top 10 most-ordered product categories in PR'))

# Add Plotly Express plots to each subplot
fig.add_trace(px.bar(SP_cat_orders, x='Product_Category', y='Total_Orders').data[0], row=1, col=1)
fig.add_trace(px.bar(RJ_cat_orders, x='Product_Category', y='Total_Orders').data[0], row=1, col=2)
fig.add_trace(px.bar(MG_cat_orders, x='Product_Category', y='Total_Orders').data[0], row=2, col=1)
fig.add_trace(px.bar(RS_cat_orders, x='Product_Category', y='Total_Orders').data[0], row=2, col=2)
fig.add_trace(px.bar(PR_cat_orders, x='Product_Category', y='Total_Orders').data[0], row=3, col=1)


# Update subplot titles (optional)
fig.update_layout(title_text='What are the top 10 most-ordered product categories in those five states?',
                  autosize=False,
                  width=1000,
                  height=1200)

# Show the plot
fig.show()